In [1]:
from mpc_nn_model_safe import *
from mpc_nn_model_exploratory import *
import mpc_nn_model_safe as mpc_safe
import mpc_nn_model_exploratory as mpc_exploratory
from platform import python_version

print(python_version())

names of files-Encoder ../trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen1_816_pred1_24_epochs200_learning_rate0.0003_hidden_states4encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included_for_results.pth 

names of files-Decoder ../trained_models/IRL_based_models/IRL_SVOpolicy_ais_pred1_24_gen1_816_epochs200_learning_rate0.0003_hidden_states4encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included_for_results.pth 

nn_prediction tensor([-60.2943,   8.5563,   1.4931, -58.6863,   8.8705,   1.3542, -56.8856,
          9.0544,   1.2097, -55.0500,   9.2054,   1.0604, -53.1999,   9.4246,
          0.9507, -51.2506,   9.5566,   0.8184, -49.4157,   9.6684,   0.6685,
        -47.5042,   9.6339,   0.5680, -45.5699,   9.8314,   0.4852, -43.5963,
          9.8108,   0.3748], grad_fn=<AddBackward0>)
nn_prediction tensor([-55.4833,   4.6515,   0.3003, -54.5884,   4.6903,   0.2940, -53.6729,
          4.7835,   0.2710, -52.6217,   4.8115,   0.2327, -51.7348,  

In [2]:
################################################################
###################### FOR SAFE POLICY #########################
####### Parameters #######
n_epochs=200
min_timesteps=50
n_rollout=10
n_skips_per_rollout=0
n_test=1
n_input=6
n_output=n_rollout*3 # 3 for each of the next n_rollout time steps
n_state_enc=4
learning_rate=0.0003
gen_model=1
pred_model=1
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

## Load the network ##
mpc_nn_predictor_safe=mpc_safe.MPC_NN_Predictor(n_epochs,min_timesteps,n_rollout,n_skips_per_rollout,n_test,n_input,n_output,n_state_enc,learning_rate,gen_model,pred_model,device)
mpc_nn_predictor_safe.load_model_weights("encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included_for_results")

###################### FOR SAFE POLICY #########################
################################################################



names of files-Encoder ../trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen1_816_pred1_24_epochs200_learning_rate0.0003_hidden_states4encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included_for_results.pth 

names of files-Decoder ../trained_models/IRL_based_models/IRL_SVOpolicy_ais_pred1_24_gen1_816_epochs200_learning_rate0.0003_hidden_states4encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included_for_results.pth 



'loaded the weights!'

In [3]:
#######################################################################
###################### FOR EXPLORATORY POLICY #########################
####### Parameters #######
n_epochs=200
min_timesteps=50
n_rollout=10
n_skips_per_rollout=0
n_test=1
n_input=6
n_output=n_rollout*3 # 3 for each of the next n_rollout time steps
n_state_enc=4
learning_rate=0.0003
gen_model=1
pred_model=1
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

## Load the network ##
mpc_nn_predictor_exploratory=mpc_exploratory.MPC_NN_Predictor(n_epochs,min_timesteps,n_rollout,n_skips_per_rollout,n_test,n_input,n_output,n_state_enc,learning_rate,gen_model,pred_model,device)
mpc_nn_predictor_exploratory.load_model_weights("encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included")

###################### FOR EXPLORATORY POLICY #########################
#######################################################################


names of files-Encoder ../trained_models/IRL_based_models/IRL_explorepolicy_ais_gen1_816_pred1_24_epochs200_learning_rate0.0003_hidden_states4encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included.pth 

names of files-Decoder ../trained_models/IRL_based_models/IRL_explorepolicy_ais_pred1_24_gen1_816_epochs200_learning_rate0.0003_hidden_states4encoder_rnn_decoder_simple_nn_n_rollout_n_skips_MSELOSS_action_t_included.pth 



'loaded the weights!'

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy.linalg as LA
from scipy.integrate import odeint
from timeit import default_timer as timer
from scipy.optimize import fmin, minimize

In [5]:
import sys
if sys.platform == 'win32' or sys.platform == 'win64': # Change to suit the OS
    sys.path.append(r"C:\Users\vietale\Documents\casadi-windows-py39-v3.5.5-64bit")
else:
    sys.path.append(r"casadiinstalldir")
import casadi as ca

In [6]:
from vehicle import *

In [46]:
class MPC:
    
    def __init__(self, H, T, bounds):
        self.H = H; self.T = T
        self.u_min = bounds['u_min']; self.u_max = bounds['u_max']
        self.v_min = bounds['v_min']; self.v_max = bounds['v_max']

    def set_state(self, st, input):
        self.input = input
        self.state = st
        
    def set_road_info(self, r):
        self.r = r
        
    def set_weight(self, W1, W12):
        self.W1 = W1; self.W12 = W12

    def compute_objective(self, PA, VA, UA, PH, VH):
        cost = 0
        for k in range(self.H):
            cost += self.W1[0]*UA[k]**2 + self.W1[1]*(VA[k] - self.v_max)**2
            cost += -self.W12*ca.log((PA[k]+0.5*VA[k])**2 + (PH[k]+0.5*VH[k])**2 + 1e-6) 

        return cost

    def formulateMPC(self):
        # Create MPC optimizer
        self.mpc_opti = ca.Opti()
        
        # Variables for CAV
        self.U = self.mpc_opti.variable(self.H)
        self.V = self.mpc_opti.variable(self.H)
        self.P = self.mpc_opti.variable(self.H)
            
        self.X0 = self.mpc_opti.parameter(2) # initial condition of CAV state
        self.PH = self.mpc_opti.parameter(self.H) # the positions of the HDV over the horizon
        self.VH = self.mpc_opti.parameter(self.H) # the speeds of the HDV over the horizon

        # Objective function
        self.J = 0
        for k in range(self.H):
            self.J += self.W1[0]*self.U[k]**2 + self.W1[1]*(self.V[k] - self.v_max)**2
            self.J += -self.W12*ca.log((self.P[k]+0.5*self.V[k])**2 + (self.PH[k]+0.5*self.VH[k])**2 + 1e-6)
            if k == 0:
                self.J += self.W1[2]*(self.U[k]- self.input)**2
            else:
                self.J += self.W1[2]*(self.U[k]- self.U[k-1])**2
        
        self.mpc_opti.minimize(self.J) 
        
        # Dynamics
        for k in range(self.H):
            if k == 0:
                self.mpc_opti.subject_to(self.P[k] == self.X0[0] + self.T*self.X0[1] \
                                            + 0.5*self.T**2*self.U[k])
                self.mpc_opti.subject_to(self.V[k] == self.X0[1] + self.T*self.U[k])            
            else:
                self.mpc_opti.subject_to(self.P[k] == self.P[k-1] + self.T*self.V[k-1] \
                                            + 0.5*self.T**2*self.U[k])
                self.mpc_opti.subject_to(self.V[k] == self.V[k-1] + self.T*self.U[k])
            
         # Constraints
        self.mpc_opti.subject_to(self.v_min <= self.V)
        self.mpc_opti.subject_to(self.V <= self.v_max)
        self.mpc_opti.subject_to(self.u_min <= self.U)
        self.mpc_opti.subject_to(self.U <= self.u_max)
        # self.D = (self.P+1.0*self.V)**2 + (self.PH+1.0*self.VH)**2 
        # self.mpc_opti.subject_to(self.r**2 <= self.D)
        
        p_opts = {'verbose_init': False, 'print_time': 0}
        s_opts = {'tol': 0.001, 'print_level': 0, 'max_iter': 100}
        self.mpc_opti.solver('ipopt', p_opts, s_opts)

        # Warm up
        self.mpc_opti.set_value(self.X0, self.state)
        self.mpc_opti.set_value(self.PH, np.zeros(self.H))
        self.mpc_opti.set_value(self.VH, np.zeros(self.H))
         
        try:
            sol = self.mpc_opti.solve()
            
        except RuntimeError:
            print("An exception occurred")
            self.mpc_opti.set_initial(self.U, self.mpc_opti.debug.value(self.U))
        else:
            self.mpc_opti.set_initial(self.U, sol.value(self.U))
            self.mpc_opti.set_initial(self.V, sol.value(self.V))
            self.mpc_opti.set_initial(self.P, sol.value(self.P))
        
    def solveMPC(self, PH, VH):
        self.mpc_opti.set_value(self.X0, self.state)
        self.mpc_opti.set_value(self.PH, PH)
        self.mpc_opti.set_value(self.VH, VH)

        try:
            sol = self.mpc_opti.solve()
        except RuntimeError:
            print("An exception occurred")
            OUT = {'P': sol.value(self.P), 'V': self.mpc_opti.debug.value(self.V), 'U': self.mpc_opti.debug.value(self.U)}
        else:
            OUT = {'P': self.mpc_opti.debug.value(self.P), 'V': sol.value(self.V), 'U': sol.value(self.U)} 
            self.mpc_opti.set_initial(self.U, np.hstack((sol.value(self.U)[1:], sol.value(self.U)[-1:])))    
            self.mpc_opti.set_initial(self.V, np.hstack((sol.value(self.V)[1:], sol.value(self.V)[-1:])))    
            self.mpc_opti.set_initial(self.P, np.hstack((sol.value(self.P)[1:], sol.value(self.P)[-1:])))    
                      
        return OUT


## Main simulation

In [61]:
def MPC_with_exploratory(p_init, v_init, W2):
    T = 0.2; H = 10; Ts = 0.05
    bounds = {'v_min' : 0.0, 'v_max' : 14.0, 'u_min' : -3.0, 'u_max' : 2.0}
    r = 10.0 

    Cars = []
    Cars.append(CAV(p_init[0], v_init[0], bounds, T, Ts))
    Cars.append(HDV(p_init[1], v_init[1], bounds, T, Ts))

    control = MPC(H, T, bounds)
    control.set_road_info(r)
    control.set_state(Cars[0].state, Cars[0].u)
    W1 = np.array([1e0,1e1,1e1])
    W2 = np.array([1e-2,1e0]); W12 = 1e3
    control.set_weight(W1, W12)
    control.formulateMPC()
    latest_observation=[Cars[0].p, Cars[1].p, Cars[0].v, Cars[1].v]
    previous_action=[Cars[0].u, Cars[1].u]
    #ais_encoder_previous=0

    ais_encoder_previous_exploratory=0

    Cars[0].const_U(0.0, 0.0)
    Cars[1].const_U(Cars[1].IRL(Cars[0].state, np.hstack([W2, W12])), 0.0)

    current_CAV_action = 0.0
    nn_prediction,ais_encoder_previous_exploratory=mpc_nn_predictor_exploratory.nn_prediction_model(latest_observation,previous_action,ais_encoder_previous_exploratory,current_CAV_action,time_step=0)
    prediction = nn_prediction.detach().cpu().numpy()
    prediction = prediction.reshape([3,-1], order = 'F')

    L = 100 # number of simulation steps
    ite = 5
    travel_time = 0
    for k in range(L):
        t = (k+1)*T
        # print("Time %s" %t)
        control.set_state(Cars[0].state, Cars[0].u)

        latest_observation=[Cars[0].p, Cars[1].p, Cars[0].v, Cars[1].v]
        previous_action=[Cars[0].u, Cars[1].u]

        COST = []
        U1 = [0.0]; current_CAV_action = 0.0
        for i in range(ite):
            nn_prediction,ais_encoder_previous_exploratory=mpc_nn_predictor_exploratory.nn_prediction_model(latest_observation,previous_action,ais_encoder_previous_exploratory,current_CAV_action,time_step=0)
            prediction = nn_prediction.detach().cpu().numpy()
            prediction = prediction.reshape([3,-1], order = 'F')
            PH = prediction[0,:]; VH = prediction[1,:]
            TRAJ = control.solveMPC(PH, VH)
            c = control.compute_objective(TRAJ['P'], TRAJ['V'], TRAJ['U'], PH, VH)
            COST.append(c)
            current_CAV_action = TRAJ['U'][0]
            U1.append(current_CAV_action)

        # print(COST, U1)
        u1 = U1[-1]
        u2 = Cars[1].IRL(Cars[0].state, np.hstack([W2, W12]))
        U = [u1, u2]
        Cars[0].const_U(u1, t)
        Cars[1].const_U(u2, t)

        if Cars[0].p > 0.0:
            travel_time = T*k
            break
        
    safe = all((Cars[0].X_hist[0,:]**2 + Cars[1].X_hist[0,:]**2) >= r**2)
    return safe, travel_time


In [62]:
def MPC_with_safe(p_init, v_init, W2):
    T = 0.2; H = 10; Ts = 0.05
    bounds = {'v_min' : 0.0, 'v_max' : 14.0, 'u_min' : -3.0, 'u_max' : 2.0}
    r = 10.0 

    Cars = []
    Cars.append(CAV(p_init[0], v_init[0], bounds, T, Ts))
    Cars.append(HDV(p_init[1], v_init[1], bounds, T, Ts))

    control = MPC(H, T, bounds)
    control.set_road_info(r)
    control.set_state(Cars[0].state, Cars[0].u)
    W1 = np.array([1e0,1e1,1e1])
    W2 = np.array([1e-2,1e0]); W12 = 1e3
    control.set_weight(W1, W12)
    control.formulateMPC()
    latest_observation=[Cars[0].p, Cars[1].p, Cars[0].v, Cars[1].v]
    previous_action=[Cars[0].u, Cars[1].u]
    #ais_encoder_previous=0

    ais_encoder_previous_safe=0

    Cars[0].const_U(0.0, 0.0)
    Cars[1].const_U(Cars[1].IRL(Cars[0].state, np.hstack([W2, W12])), 0.0)

    current_CAV_action = 0.0
    nn_prediction,ais_encoder_previous_safe=mpc_nn_predictor_safe.nn_prediction_model(latest_observation,previous_action,ais_encoder_previous_safe,current_CAV_action,time_step=0)
    prediction = nn_prediction.detach().cpu().numpy()
    prediction = prediction.reshape([3,-1], order = 'F')

    L = 100 # number of simulation steps
    ite = 5
    travel_time = 0
    for k in range(L):
        t = (k+1)*T
        # print("Time %s" %t)
        control.set_state(Cars[0].state, Cars[0].u)

        latest_observation=[Cars[0].p, Cars[1].p, Cars[0].v, Cars[1].v]
        previous_action=[Cars[0].u, Cars[1].u]

        COST = []
        U1 = [0.0]; current_CAV_action = 0.0
        for i in range(ite):
            nn_prediction,ais_encoder_previous_safe=mpc_nn_predictor_safe.nn_prediction_model(latest_observation,previous_action,ais_encoder_previous_safe,current_CAV_action,time_step=0)
            prediction = nn_prediction.detach().cpu().numpy()
            prediction = prediction.reshape([3,-1], order = 'F')
            PH = prediction[0,:]; VH = prediction[1,:]
            TRAJ = control.solveMPC(PH, VH)
            c = control.compute_objective(TRAJ['P'], TRAJ['V'], TRAJ['U'], PH, VH)
            COST.append(c)
            current_CAV_action = TRAJ['U'][0]
            U1.append(current_CAV_action)

        # print(COST, U1)
        u1 = U1[-1]
        u2 = Cars[1].IRL(Cars[0].state, np.hstack([W2, W12]))
        U = [u1, u2]
        Cars[0].const_U(u1, t)
        Cars[1].const_U(u2, t)

        if Cars[0].p > 0.0:
            travel_time = T*k
            break
        
    safe = all((Cars[0].X_hist[0,:]**2 + Cars[1].X_hist[0,:]**2) >= r**2)
    return safe, travel_time

In [66]:
# This loop is to run multiple simulations to get statistics
np.random.seed(28)
Safe_stats = {"Safe": [], "Time": []}
Expl_stats = {"Safe": [], "Time": []}
for k in range(5):
    print(("Simulation %s" %(k+1)))
    p_init = np.random.uniform(low = -80, high = -60, size = 2)
    v_init = np.random.uniform(low = 10, high = 14, size = 2)
    W2 = 10**np.random.uniform(low = -2, high = 2, size = 2)
    s, t = MPC_with_safe(p_init, v_init, W2)
    Safe_stats["Safe"].append(s); Safe_stats["Time"].append(t)
    s, t = MPC_with_exploratory(p_init, v_init, W2)
    Expl_stats["Safe"].append(s); Expl_stats["Time"].append(t)


Simulation 1
Simulation 2
Simulation 3
Simulation 4
Simulation 5


In [67]:
Expl_stats

{'Safe': [True, True, False, True, True],
 'Time': [6.6000000000000005, 7.0, 5.0, 7.4, 6.800000000000001]}

In [68]:
Safe_stats

{'Safe': [True, True, True, True, True],
 'Time': [6.0, 5.800000000000001, 6.6000000000000005, 6.2, 5.6000000000000005]}